# "Live" Market USD Swaps Pricer
- Using Eris' Discount Curve: https://www.erisfutures.com/forum/ftp-site-guide

In [3]:
import sys
sys.path.append("../")

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

In [11]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from core.utils.ql_loader import ql

from core.IRSwaps import IRSwaps, IRSwapQuery, IRSwapStructure, IRSwapStructureFunctionMap

import nest_asyncio
nest_asyncio.apply()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
curve = "USD-SOFR-1D"

usd_ois = IRSwaps(
    curve=curve,
    data_source="CME",
    ql_interpolation_algo="mono_log_parabolic_cubic",
    max_njobs=-1,
)

In [9]:
ts, ql_curve = usd_ois.fetch_latest_curve()
curve_handle = ql.YieldTermStructureHandle(ql_curve)

ts, ql_curve

FETCHING ERIS INTRADAY DISC CURVE...: 100%|██████████| 1/1 [00:00<00:00,  2.27it/s]


(datetime.datetime(2025, 5, 12, 18, 33, 19, tzinfo=<UTC>),
 <QuantLib.QuantLib.MonotonicLogParabolicCubicDiscountCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedDiscountCurve< MonotonicLogParabolicCubic > > *' at 0x000001EFD32A2F70> >)

## "Live" Level for an Outright Rec 100k/bp in spot 10s

In [10]:
q = IRSwapQuery(tenor="10Y", structure=IRSwapStructure.OUTRIGHT, structure_kwargs={"bpv": 100_000})
package, risk_weights = IRSwapStructureFunctionMap(curve=curve, curve_handle=curve_handle).apply(structure=q.structure, tenor=q.tenor, **q.structure_kwargs)

outright: ql.OvernightIndexedSwap = package[0]

print("Eff/Mat Dates ", outright.startDate(), outright.maturityDate())
print("Par Rate: ", outright.fairRate() * 100)
print("Notional: ", outright.nominal())
print("NPV: ", outright.NPV())
print("BPV: ", outright.fixedLegBPS())

Eff/Mat Dates  May 14th, 2025 May 14th, 2035
Par Rate:  3.942955797694999
Notional:  121005521.44907977
NPV:  4.470348358154297e-08
BPV:  100000.0
